In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [2]:
client_id = 'e738e985fed449e8846df5ee72139b87'
client_secret = '' 
# invalidated the previous one

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_token_headers(self):
        client_creds = f"{self.client_id}:{self.client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return {"Authorization": f"Basic {client_creds_b64.decode()}"}
    
    def get_token_data(self):
        return {"grant_type": "client_credentials"}
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            return False
        token_response_data = r.json()
        now = datetime.datetime.now()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        auth_done = self.perform_auth()
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, _id, resource_type="albums"):
        endpoint = f"https://api.spotify.com/v1/{resource_type}/{_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'):
        headers = self.get_resource_header()
        url = "https://api.spotify.com/v1/search"
        data = urlencode({
            "q": "Blank Space", 
            "type": "track"
        })

        lookup_url = f"{url}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [4]:
client = SpotifyAPI(client_id, client_secret)

In [ ]:
client.search("Time", search_type="Track")

In [ ]:
client.get_artist("06HL4z0CvFAxyc27GXpf02")

In [ ]:
client.get_album("2QJmrSgbdM35R67eoGQo4j")

In [8]:
url = "https://accounts.spotify.com/authorize?"
headers = urlencode({
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://127.0.0.1:5000/callback"
})
new_url = f'{url}{headers}'
realHeaders = client.get_resource_header()
r = requests.get(new_url, realHeaders)

In [9]:
print(r.text)


<!DOCTYPE html>
<html id="app" lang="en" dir="ltr" ng-csp ng-strict-di>
<head>
  <meta charset="utf-8">
  <title ng-bind="(&#39;loginTitle&#39; | localize) + &#39; - Spotify&#39;">Spotify</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  <base href="/">
  <link rel="icon" href="https://accounts.scdn.co/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">
  <link href="https://accounts.scdn.co/css/index.843d063c7d7c72e443ef.css" media="screen" rel="stylesheet">

  <script defer src="https://accounts.scdn.co/js/index.843d063c7d7c72e443ef.js" sp-bootstrap></script>
  <meta ng-non-bindable sp-bootstrap-data='{"phoneFeatureEnabled":false,"previewEnabled":false,"user":false,"tpaState":"AQCZOLe5o4ig5VlNy5zEgdTv2s4Gbwj5Eyl8j7qz8z7knaeiEQ5hpiYR/YapprRtl4rGuwiM7crgDY+uX4Msdigh+rKYbEuwwQL+4TB7rIZ87vEiqUGMCFy9v48q5/a3jODARHclo4HQBxCtUvkl1rCMyW5oGzlOB1pjGZXJWZhkjffPBRzc/8eeYdMLB9rNfNhXR7OsFgpJ0UfwuBlj4VBUfsj0EDhekP0DDDJx8+Utqb6

In [33]:
auth_code = 'AQBE2LsoIEsWiBMjEBgh16BWAdUf-FGBUmmPMHsIVe6Mui8VO6UQcDm0-4wRJGG4UXqYu-lmhmk-nVV7QXeWH8v4A4jjD-vjbJCXQXNpznkO6Td3v5ttCy39B2YRbGBWKwNGawyBQ3d2L0Jv9ZiBI71MNHnbeoA65GYggpEY801qGQ'
data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': 'http://127.0.0.1:5000/callback'
}
headers = client.get_token_headers()
r = requests.post(url, data=data, headers=headers)

In [38]:
test = r.json()
print(test)

{'access_token': 'BQC0kOfZw-yFRJarecDED3WK8o94DiPekyItXHAakWJpxKITLKVcS-2Jb0rK8PIwjZM4kwACsfuyoW1Yweo8VlbzEsfcNzNDliDKOIpG1zvfB16Ktk362oy5tteVSSF9ohKDtwsQlkS0Itf-lK8q', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQAIHzkgowo8IX757yqUYMlqVniXkfbwQ12r4TFDT30VjMQY7WmyHQXbHTA7rWmy7uiX-hya7UR7lf6czUDPRPwQhSEqS5gVulxj9H6Lw1YXRRebNEeFr8ES0mqrytXIYz8', 'scope': ''}


In [15]:
url = 'https://accounts.spotify.com/api/token'
refresh_token = 'AQAGPJDESNsvAu0k7fw2c1RFd07YGpyGyJkxOQmkoB0BEhacgwlDcUDVMeI6240LWgb4Ed21fnWofVmBfyJSsyMYgp5ukv_go6dUB1HD3FRi65qO29dIAvHdWFdMAp8nVEI'
data = {
    'grant_type': 'refresh_token',
    'refresh_token': refresh_token,
}
headers = client.get_token_headers()
r = requests.post(url, data=data, headers=headers)

In [16]:
print(r.text)

{"access_token":"BQCK9jLn4lKb-5H1ouuZ0X0Z-KtO4o-smpnTMJ5j2pyqMnSRihhY28pIZQApuSc70J1mWSdTLrZcu_cd7NG7hfVF2FVO_QZWT6ivunwDQrdjW1Cth0LrKsmOv3WwuZjo44YzGwrXw99oiZwV8VWzFHBsuwgl","token_type":"Bearer","expires_in":3600,"scope":"user-top-read"}


In [13]:
url = "https://api.spotify.com/v1/me/top/tracks"
access_token = 'BQA8heRmBNNVdkVfrkZwzrxeZUyBL416gw7Sc7cIWZG-XVI8QRDTj5CXCK9c7wRV5W6cfncmD1WSVdVug2o5meoQbngQtl_TMnQueu3cPWwAxQLk1BmrR6ZC6rqcoZcrXlx2WapZKmERu205nWbypvdwdJ2Z'
headers = {
    'Authorization': f'Bearer {access_token}',
}
r = requests.get(url, headers=headers)
print(r)

<Response [200]>


In [14]:
print(r.text)

{
  "items" : [ {
    "album" : {
      "album_type" : "SINGLE",
      "artists" : [ {
        "external_urls" : {
          "spotify" : "https://open.spotify.com/artist/3JDG63cSaK3xgDnB2H55Xp"
        },
        "href" : "https://api.spotify.com/v1/artists/3JDG63cSaK3xgDnB2H55Xp",
        "id" : "3JDG63cSaK3xgDnB2H55Xp",
        "name" : "Audrey Mika",
        "type" : "artist",
        "uri" : "spotify:artist:3JDG63cSaK3xgDnB2H55Xp"
      } ],
      "available_markets" : [ "AD", "AE", "AR", "AT", "AU", "BE", "BG", "BH", "BO", "BR", "CA", "CH", "CL", "CO", "CR", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HU", "ID", "IE", "IL", "IN", "IS", "IT", "JO", "JP", "KW", "LB", "LI", "LT", "LU", "LV", "MA", "MC", "MT", "MX", "MY", "NI", "NL", "NO", "NZ", "OM", "PA", "PE", "PH", "PL", "PS", "PT", "PY", "QA", "RO", "SA", "SE", "SG", "SK", "SV", "TH", "TN", "TR", "TW", "US", "UY", "VN", "ZA" ],
      "external_urls" : {
        "spotify" 